In [28]:
from datetime import datetime
import heapq

In [29]:
class FoodItem:
    def __init__(self, name, expiry, nutrients, quantity): 
        self.name = name 
        self.expiry = expiry
        self.nutrients = nutrients
        self.quantity = quantity

    def setID(self, id): 
        self.itemID = id



In [30]:
class Inventory:
    def __init__(self): 
        self.items = {}
    def add_item(name, id, expiry, nutrients, quantity):
        expiry = datetime.strptime(expiry, "%Y-%m-%d")
        newItem = FoodItem(name, id, expiry, nutrients, quantity)
    def search_item(self, name):
        name = name.upper()
        if self.items != None:
            for indiv_item in self.items.keys():
                if indiv_item.upper() == name: 
                    return self.items[name]




In [31]:

print("Choose an action:")
print("---------------------")
inventory = Inventory()
while True: 
    inp = int(input("1. Add an item \n 2.Search an item \n 3.Delete an item"))
    if inp == 1:
        name =  input("Enter the name of the food item:")
        expiry = input("Enter expiry date (YYYY-MM-DD): ")
        quantity = int(input("Enter quantity:"))
        print("Enter nutritional info:")
        calories = int(input("  Calories: "))
        protein = int(input("  Protein (grams): ")) 
        nutrients = {"calories": calories, "protein": protein}
        inventory.add_item(name, expiry, nutrients, quantity)
        print(f"✅ Added {name}!\n")
        break
        

Choose an action:
---------------------
Enter nutritional info:


ValueError: time data '10, 11, 05' does not match format '%Y-%m-%d'